# Bibliotecas

In [ ]:
import numpy as np
from array import array

# Classe Pai

In [ ]:
class EDL:
    def __init__(self, matrix):
        self.matrix = matrix
    def imprimir(self):
        for row in self.matrix:
            print(row)
    
    def __str__(self):
        """ Retorna uma representação formatada da matriz. """
        matrix_str = ""
        for linha in self.matrix:
            matrix_str += "|"
            for j in linha:
                matrix_str += f" {j:^5}"  # Adiciona elementos formatados à string.
            matrix_str += " |\n"
        return matrix_str
    
    def __add__(self,other):
        """ Soma duas matrizes. """
        M1 = len(self.matrix) ; M2 = len(other.matrix)
        N1 = len(self.matrix[0]) ; N2 = len(other.matrix[0])
        if M1 != M2 or N1 != N2:
            raise ValueError("As matrizes devem ter as mesmas dimensões para a soma.")
        resultado = np.zeros((M1,N1))
        for i,_ in enumerate(self.matrix):
            for j,_ in enumerate(self.matrix[i]):
                soma_elemento = self.matrix[i][j] + other.matrix[i][j] 
                resultado[i][j] = soma_elemento
        return EDL(resultado)
    
    def __sub__(self,other):
        """ Subtrai duas matrizes. """
        M1 = len(self.matrix) ; M2 = len(other.matrix)
        N1 = len(self.matrix[0]) ; N2 = len(other.matrix[0])
        if M1 != M2 or N1 != N2:
            raise ValueError("As matrizes devem ter as mesmas dimensões para a soma.")
        resultado = np.zeros((M1,N1))
        for i,_ in enumerate(self.matrix):
            for j,_ in enumerate(self.matrix[i]):
                soma_elemento = self.matrix[i][j] - other.matrix[i][j] 
                resultado[i][j] = soma_elemento
        return EDL(resultado)

    def __mul__(self,other):
        """ Multiplica duas matrizes ou multiplica uma matriz por um escalar. """
        if isinstance(other, (int, float)):
            # Se o segundo operando for um número, multiplica todos os elementos da matriz pelo número
            resultado = [[elemento * other for elemento in linha] for linha in self.matrix]
            return EDL(resultado)
        M1 = len(self.matrix) ; M2 = len(other.matrix)
        N1 = len(self.matrix[0]) ; N2 = len(other.matrix[0])
        if M1 != M2 or N1 != N2:
            raise ValueError("As matrizes devem ter as mesmas dimensões para a soma.")
        resultado = []
        for i in range(len(self.matrix)):
            linha = []
            for j in range(len(other.matrix[0])):
                produto = 0
                for k in range(len(self.matrix[0])):
                    produto += self.matrix[i][k] * other.matrix[k][j]
                linha.append(produto)
            resultado.append(linha)
        return EDL(resultado)
    
    def __rmul__(self,other):
        """ Multiplica uma matriz por um escalar, se "other" for uma matriz,
        o método __mul__ será utilizado. """
        if isinstance(other, (int, float)):
            resultado = [[elemento * other for elemento in linha] for linha in self.matrix]
        return EDL(resultado)

# Sub classe EDL - Quadrada

In [ ]:
class Quadrada(EDL):
    def __init__(self, matrix):
        super().__init__(matrix)
    
    def __sub__(self,other):
        """ Subtrai duas matrizes Quadradas. """
        return super().__sub__(other)
    
    def __add__(self,other):
        """ Soma duas matrizes Quadradas. """
        return super().__add__(other)
    
    def __mul__(self,other):
        """ Multiplica duas matrizes Quadradas ou multiplica uma matriz Quadrada por um escalar. """
        return super().__mul__(other)
    
    def __rmul__(self,other):
        """ Multiplica uma matriz Quadrada por um escalar, se "other" for uma matriz,
        o método __mul__ será utilizado. """
        return super().__rmul__(other)
    
    def traco(self):
        """ Calcula o traço da matriz (soma dos elementos na diagonal principal). """
        if len(self.matrix) != len(self.matrix[0]):
            raise ValueError("O traço só pode ser calculado para matrizes quadradas.")
        traco = sum(self.matrix[i][i] for i in range(len(self.matrix)))
        return traco
    
    def transposta(self):
        """ Retorna a transposta da matriz. """
        num_linhas = len(self.matrix)
        num_colunas = len(self.matrix[0])
        transposta_matrix = [[0.0] * num_linhas for _ in range(num_colunas)]
        for i in range(num_linhas):
            for j in range(num_colunas):
                transposta_matrix[j][i] = self.matrix[i][j]
        return Quadrada([array('d', linha) for linha in transposta_matrix])
    
    def determinante(self):
        """ Calcula o determinante da matriz. """
        def calcular_cofator(linha, coluna, matriz):
            """ Calcula o cofator da matriz. """
            submatriz = [linha[0:coluna] + linha[coluna + 1:] for linha in matriz[1:]]
            submatriz_quadrada = Quadrada(submatriz)
            return (-1) ** (linha + coluna) * submatriz_quadrada.determinante()
        if len(self.matrix) == 1:
            return self.matrix[0][0]
        if len(self.matrix) == 2:
            # Caso base para matrizes 2x2:
            return self.matrix[0][0] * self.matrix[1][1] - self.matrix[0][1] * self.matrix[1][0]
        det = 0
        for coluna in range(len(self.matrix)):
            cofator = calcular_cofator(0, coluna, self.matrix)
            det += self.matrix[0][coluna] * cofator
        return Quadrada(det)

# Sub classe EDL - Triangulares Superior:


In [ ]:
class TriangularSuperior(EDL):
    def __init__(self, matrix):
        super().__init__(matrix)
    
    def eh_triangular_superior(self):
        """ Verifica se a matriz é triangular superior. """
        for i in range(len(self.matrix)):
            for j in range(i):
                if self.matrix[i][j] != 0:
                    return False
        return True
        
    def __add__(self, other):
        """ Soma duas matrizes triangulares superiores.
        Caso uma delas não seja superior, usa a adição da classe EDL. """
        if not self.eh_triangular_superior() or not other.eh_triangular_superior():
            return super().__add__(other)
        resultado = [[self.matrix[i][j] + other.matrix[i][j] if j >= i else 0 for j in range(len(self.matrix[i]))] for i in range(len(self.matrix))]
        return TriangularSuperior(resultado)
    
    def __sub__(self, other):
        """ Subtrai duas matrizes triangulares superiores.
        Caso uma não seja superior, usa a subtração da classe EDL. """
        if not self.eh_triangular_superior() or not other.eh_triangular_superior():
            return super().__sub__(other)
        resultado = [[self.matrix[i][j] - other.matrix[i][j] if j >= i else 0 for j in range(len(self.matrix[i]))] for i in range(len(self.matrix))]
        return TriangularSuperior(resultado)
    
    def __mul__(self, other):
        """ Multiplica duas matrizes triangulares superiores. """
        if not self.eh_triangular_superior() or not isinstance(other, TriangularSuperior):
            raise ValueError("Ambas as matrizes devem ser triangulares superiores.")
        if len(self.matrix) != len(other.matrix):
            raise ValueError("O número de linhas das matrizes deve ser igual para a multiplicação.")
        resultado = [[sum(self.matrix[i][k] * other.matrix[k][j] for k in range(j, len(other.matrix[i]))) if j >= i else 0 for j in range(len(other.matrix[i]))] for i in range(len(self.matrix))]
        return TriangularSuperior(resultado)

    def __rmul__(self, other):
        return super().__rmul__(other)
    
    def determinante(self):
        """ Calcula o determinante da matriz triangular superior. """
        if len(self.matrix) == 1:
            return self.matrix[0][0]
        det = 1
        for i in range(len(self.matrix)):
            det *= self.matrix[i][i]
        return det
    
    def inversa(self):
        """ Calcula a inversa da matriz triangular superior. """
        if not self.eh_triangular_superior():
            raise ValueError("A matriz não é triangular superior e, portanto, não possui inversa.")
        inversa = [[1/self.matrix[i][j] if j == i else 0 for j in range(len(self.matrix[i]))] for i in range(len(self.matrix))]
        return TriangularSuperior(inversa)



# Sub classe EDL - Triangulares Inferior:

In [ ]:
class TriangularInferior(EDL):
    def __init__(self, matrix):
        super().__init__(matrix)
    
    def eh_triangular_inferior(self):
        """ Verifica se a matriz é triangular inferior. """
        for i in range(len(self.matrix)):
            for j in range(i + 1, len(self.matrix)):
                if self.matrix[i][j] != 0:
                    return False
        return True
        
    def __add__(self, other):
        """ Soma duas matrizes triangulares inferiores.
        Caso uma delas não seja inferior, usa a adição da classe EDL. """
        if not self.eh_triangular_inferior() or not other.eh_triangular_inferior():
            return super().__add__(other)
        resultado = [[self.matrix[i][j] + other.matrix[i][j] if j <= i else 0 for j in range(len(self.matrix[i]))] for i in range(len(self.matrix))]
        return TriangularInferior(resultado)
    
    def __sub__(self, other):
        """ Subtrai duas matrizes triangulares inferiores.
        Caso uma não seja inferior, usa a subtração da classe EDL. """
        if not self.eh_triangular_inferior() or not other.eh_triangular_inferior():
            return super().__sub__(other)
        resultado = [[self.matrix[i][j] - other.matrix[i][j] if j <= i else 0 for j in range(len(self.matrix[i]))] for i in range(len(self.matrix))]
        return TriangularInferior(resultado)
    
    def __mul__(self, other):
        """ Multiplica duas matrizes triangulares inferiores. """
        if not self.eh_triangular_inferior() or not isinstance(other, TriangularInferior):
            raise ValueError("Ambas as matrizes devem ser triangulares inferiores.")
        if len(self.matrix) != len(other.matrix):
            raise ValueError("O número de linhas das matrizes deve ser igual para a multiplicação.")
        resultado = [[sum(self.matrix[i][k] * other.matrix[k][j] for k in range(i + 1)) if j <= i else 0 for j in range(len(other.matrix[i]))] for i in range(len(self.matrix))]
        return TriangularInferior(resultado)

    def __rmul__(self, other):
        return super().__rmul__(other)
    
    def determinante(self):
        """ Calcula o determinante da matriz triangular inferior. """
        if len(self.matrix) == 1:
            return self.matrix[0][0]
        det = 1
        for i in range(len(self.matrix)):
            det *= self.matrix[i][i]
        return det
    
    def inversa(self):
        """ Calcula a inversa da matriz triangular inferior. """
        if not self.eh_triangular_inferior():
            raise ValueError("A matriz não é triangular inferior e, portanto, não possui inversa.")
        inversa = [[1/self.matrix[i][j] if j == i else 0 for j in range(len(self.matrix[i]))] for i in range(len(self.matrix))]
        return TriangularInferior(inversa)

# Menu

In [ ]:
def criar_matriz():
    # Função para criar uma matriz a partir das entradas do usuário
    print("Digite as dimensões da matriz (linhas x colunas):")
    num_linhas = int(input("Número de linhas: "))
    num_colunas = int(input("Número de colunas: "))

    matriz = []
    for i in range(num_linhas):
        linha = []
        print(f"Digite os elementos da linha {i + 1}, separados por espaço:")
        elementos = input().split()
        if len(elementos) != num_colunas:
            raise ValueError("O número de elementos na linha não corresponde ao número de colunas da matriz.")
        for elemento in elementos:
            linha.append(float(elemento))
        matriz.append(array('d', linha))

    return Quadrada(matriz)

def criar_lista_de_matrizes():
    lista_matrizes = []
    while True:
        matriz = criar_matriz()
        lista_matrizes.append(matriz)
        continuar = input("Deseja criar outra matriz? (S/N): ").strip().upper()
        if continuar != "S":
            break
    return lista_matrizes

def zerar_lista_de_matrizes(lista_matrizes):
    lista_matrizes.clear()
    print("A lista de matrizes foi zerada.")

def main():
    lista_matrizes = []
    while True:
        print("\nCalculadora de Matrizes")
        print("1. Criar Única Matriz")
        print("2. Criar Lista de Matrizes")
        print("3. Criar Matriz Identidade")
        print("4. Somar Duas Matrizes da Lista")
        print("5. Zerar Lista de Matrizes")
        print("6. Sair")
    

        escolha = input("Escolha uma opção: ").strip()

        if escolha == "1":
            matriz = criar_matriz()
            print("Matriz criada:")
            print(matriz)
        elif escolha == "2":
            lista_matrizes = criar_lista_de_matrizes()
            print("Lista de Matrizes:")
            for i, matriz in enumerate(lista_matrizes, start=1):
                print(f"Matriz {i}:")
                print(matriz)
        elif escolha == "3":
            matriz_identidade = criar_matriz_identidade()
            print("Matriz Identidade criada:")
            print(matriz_identidade)
            lista_matrizes.append(matriz_identidade)
        elif escolha == "4":
            if len(lista_matrizes) < 2:
                print("É necessário ter pelo menos duas matrizes na lista para realizar a soma.")
            else:
                print("Escolha duas matrizes da lista para somar:")
                print("Índices disponíveis:")
                for i, matriz in enumerate(lista_matrizes):
                    print(f"{i+1}: Matriz de ordem {len(matriz.matrix)}")
                indice1 = int(input("Digite o índice da primeira matriz: ")) -1
                indice2 = int(input("Digite o índice da segunda matriz: ")) -1
                if 0 <= indice1 < len(lista_matrizes) and 0 <= indice2 < len(lista_matrizes):
                    resultado_soma = lista_matrizes[indice1] + lista_matrizes[indice2]
                    print("Resultado da soma:")
                    print(resultado_soma)
                else:
                    print("Índices inválidos.")
        elif escolha == "5":
            zerar_lista_de_matrizes(lista_matrizes)
        elif escolha == "6":
            print("Saindo...")
            break
        else:
            print("Opção inválida. Tente novamente.")

In [ ]:
if __name__ == "__main__":
    main()